In [1]:
#parameters

root_path="C:\\Users\\bruno\\Desktop\\Diplomski\\ \\"

In [2]:
import pandas as pd
import numpy as np
import csv
import mne
from scipy.signal import find_peaks
from matplotlib import pyplot as plt, dates as mdates
import math
from datetime import datetime, timedelta, time
import pyarrow.feather as feather
from getEventTypes import getEventTypes

In [3]:
edf_path = root_path + "edf_signals.feather"
meas_date_path = root_path + "meas_date.feather"
hypnogram_path = root_path + "csv_hypnogram.csv"
events_path = root_path + "csv_events.csv"

In [4]:
table_data = feather.read_table(edf_path)
signal_no_sleep_no_events = table_data.to_pandas()

meas_table_data = feather.read_table(meas_date_path)
meas_date = meas_table_data.to_pandas()['meas_date'][0]

In [7]:
yesterday = datetime.now() - timedelta(days=1)
start_time = meas_date.replace(tzinfo=None).replace(year=yesterday.year, month=yesterday.month, day=yesterday.day)
end_time = start_time + timedelta(milliseconds=signal_no_sleep_no_events.shape[0]*2)

In [29]:
number_of_samples = signal_no_sleep_no_events.shape[0]

In [30]:
def shiftTimestampToYesterday(value):
    noon = time(12, 0)
    if(noon<value.time()):
        return value - timedelta(days=1)
    else:
        return value

SleepPhases = {
    'WK' : 0,
    'N1' : 1,
    'N2' : 2,
    'N3' : 3,
    'REM' : 4}

In [31]:
hypno_df = pd.read_csv(hypnogram_path)
events_df = pd.read_csv(events_path, sep=';')

In [32]:
hypno_df['Sleep Stage Int'] = hypno_df['Sleep Stage'].map(SleepPhases.get)
events_df['Time'] = pd.to_datetime(events_df['Time'])
events_df['Time'] = events_df['Time'].apply(shiftTimestampToYesterday) 

C:\Users\bruno\AppData\Local\Temp\ipykernel_6744\2306849214.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events_df['Time'] = pd.to_datetime(events_df['Time'])


In [33]:
hypno_df.rename(columns={' Epoch Number ':'Epoch','Sleep Stage Int': 'Stage','Sleep Stage': 'Stage Name' , 'Start Time ':'Start'}, inplace=True)
hypno_df['Start'] = pd.to_datetime(hypno_df['Start'])
hypno_df['Epoch'] = hypno_df['Epoch'].astype(int)
hypno_df['Start'] = hypno_df['Start'].apply(shiftTimestampToYesterday)
hypno_df = hypno_df.sort_values(by=['Start'])

C:\Users\bruno\AppData\Local\Temp\ipykernel_6744\1060567551.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hypno_df['Start'] = pd.to_datetime(hypno_df['Start'])


In [34]:
events_df = events_df[(events_df['Time'] >= start_time) & (events_df['Time'] <= end_time)]
hypno_df = hypno_df[(hypno_df['Start'] >= start_time) & (hypno_df['Start'] <= end_time)]

In [35]:
continuous_time_series = pd.DataFrame(pd.date_range(start=start_time, periods = number_of_samples, freq='2ms'), columns=['timestamp'])

In [36]:
event_types = getEventTypes()

for event_type in event_types:
    event_type_rows = events_df[events_df['Type'] == event_type]

    for _, row in event_type_rows.iterrows():
        start_idx = int((row['Time'] - start_time).total_seconds() * 500)
        end_idx = start_idx + int(round(float(row['Duration'].replace(',','.')) * 500))
        continuous_time_series.loc[start_idx:end_idx, event_type] = True
        
    if len(event_type_rows)==0 :
        continuous_time_series[event_type] = False

continuous_time_series = continuous_time_series.fillna(False)

In [37]:
hypno_df.set_index('Start', inplace=True)
new_frequency = '2L'

hypno_df_resampled = hypno_df.resample(new_frequency)
hypno_df_resampled_interpolated = hypno_df_resampled.interpolate()
hypno_df_resampled_interpolated.reset_index(inplace=True)
hypno_df_resampled_interpolated['Epoch'] = hypno_df_resampled_interpolated['Epoch'].apply(round)
hypno_df_resampled_interpolated['Stage'] = hypno_df_resampled_interpolated['Stage'].apply(round)
hypno_df_resampled_interpolated

last_hypno_df_row = hypno_df_resampled_interpolated.iloc[-1]

In [38]:
hypno_df_resampled_interpolated = pd.concat([hypno_df_resampled_interpolated,pd.DataFrame([last_hypno_df_row] * (number_of_samples-len(hypno_df_resampled_interpolated)))], axis=0)

In [39]:
signal_to_filter = pd.merge(signal_no_sleep_no_events, hypno_df_resampled_interpolated, left_index=True, right_index=True)
signal_to_filter = pd.merge(signal_to_filter, continuous_time_series, left_index=True, right_index=True)
signal = signal_to_filter

In [40]:
feather.write_feather(signal, root_path + "signal_sync_EDF.feather")
%time

CPU times: total: 0 ns
Wall time: 0 ns
